In [56]:
import pandas as pd

In [57]:
df = pd.read_excel('./통합 문서1.xlsx')

In [58]:
df['월'] = df['columns'].apply(lambda x : x[:3])

In [59]:
df['columns']

0         1월       상품군_가전: 78.02976067131303
1      1월       상품군_건강기능: 32.348024079406265
2        1월       상품군_농수축: 38.56661837187605
3      1월       상품군_생활용품: 27.877970915854323
4          1월       상품군_속옷: 41.1314742674058
                       ...                  
551       12월      상품군_의류: 81.97757593626386
552     12월      상품군_이미용: 103.45583925460724
553       12월      상품군_잡화: 81.02022852698389
554       12월      상품군_주방: 42.93528019641724
555        12월      test: 61.658125506464025
Name: columns, Length: 556, dtype: object

In [60]:
df['MAPE'] = df['columns'].apply(lambda x : x.split(':')[1].strip())

In [61]:
df['상품군'] = df['columns'].apply(lambda x : x.split('월')[1].split(':')[0].strip())

In [62]:
df['MAPE'] = df['MAPE'].astype(float)

In [75]:
df['version'] = None

In [86]:
df.loc[:138, 'version'] = 1

In [91]:
df.loc[139:277, 'version'] = 2

In [103]:
df.loc[278:416, 'version'] = 3

In [105]:
df.loc[417:]

,columns,월,MAPE,상품군,version
417,1월 상품군_가전: 78.63028572067249,1월,78.630286,상품군_가전,4
418,1월 상품군_건강기능: 33.71195676134861,1월,33.711957,상품군_건강기능,4
419,1월 상품군_농수축: 40.56698360222816,1월,40.566984,상품군_농수축,4
420,1월 상품군_생활용품: 25.20334069834636,1월,25.203341,상품군_생활용품,4
421,1월 상품군_속옷: 47.17197623036512,1월,47.171976,상품군_속옷,4
...,...,...,...,...,...
551,12월 상품군_의류: 81.97757593626386,12월,81.977576,상품군_의류,4
552,12월 상품군_이미용: 103.45583925460724,12월,103.455839,상품군_이미용,4
553,12월 상품군_잡화: 81.02022852698389,12월,81.020229,상품군_잡화,4
554,12월 상품군_주방: 42.93528019641724,12월,42.935280,상품군_주방,4


In [114]:
df.loc[df['상품군'] != 'test'].groupby(['version'])['MAPE'].mean().reset_index()

,version,MAPE
0,1,50.017160
1,2,49.466147
2,3,47.749564
3,4,48.468935


In [54]:
df.loc[df['상품군'] == '상품군_전체', 'MAPE'].mean()

49.9303457723808

In [115]:
df.groupby(['version', '상품군'])['MAPE'].mean()

version  상품군     
1        test         49.788737
         상품군_가구       59.668963
         상품군_가전       59.922136
         상품군_건강기능     34.181254
         상품군_농수축      26.695612
         상품군_생활용품     64.096558
         상품군_속옷       50.990215
         상품군_의류       45.434186
         상품군_이미용      32.470654
         상품군_잡화       49.232599
         상품군_주방       41.476794
         상품군_침구      105.227577
2        test         49.262480
         상품군_가구       60.083007
         상품군_가전       58.870308
         상품군_건강기능     34.339319
         상품군_농수축      27.000153
         상품군_생활용품     63.793556
         상품군_속옷       50.731024
         상품군_의류       44.841101
         상품군_이미용      34.078091
         상품군_잡화       51.675916
         상품군_주방       40.298571
         상품군_침구       94.218897
3        test         47.601211
         상품군_가구       60.066946
         상품군_가전       58.447025
         상품군_건강기능     32.498678
         상품군_농수축      26.032851
         상품군_생활용품     63.051702
         상품군_속옷       